<a href="https://colab.research.google.com/github/njlauersdorf/personal_projects/blob/main/User_Friendly_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.linear_model import LinearRegression

# Sklearn Packages
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import skew

# Sklearn Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error, precision_score, confusion_matrix, accuracy_score

# Visualizes all the columns
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

#Import DF
df = input("Enter the path to the CSV file: ")
df = pd.read_csv(df)

#Renaming Columns for Better Understanding
df = df.rename(columns={'tid':'Tess ID',
'st_pmra':'stellar_ra',
'st_pmdec':'stellar_dec',
'pl_tranmid':'planet_transit_midpoint[BJD]',
'pl_orbper':'planet_orbit_period[days]',
'pl_trandurh':'planet_transit_duration[hours]',
'pl_trandep':'planet_transit_depth[ppm]',
'pl_rade':'Exoplanet Radii (EAU)',
'pl_insol':'planet_insolation[Earth_flux]',
'pl_eqt':'planet_equilibrium_temperature[K]',
'st_tmag':'stellar_tess_magnitude',
'st_dist':'stellar_distance[pc]',
'st_teff':'Stellar Temperature',
'st_logg':'stellar_gravity',
'st_rad':'stellar_radius[R_sun]',
'rowupdate':'date_modified',
})

# Renaming TIC
df["Tess ID"] = 'TIC ' + df["Tess ID"].astype(str)

#Calculating light years
df['stellar_distance[ly]']= df['stellar_distance[pc]'] * 3.26156378

#Dropping NAN Values
df = df.dropna(subset=['Exoplanet Radii (EAU)','stellar_radius[R_sun]',
                       'Stellar Temperature', 'planet_orbit_period[days]'])

# Assigning Habitability Disposition
df['Disposition'] = ['1' if .5<=x<1.544 else '2' if 1.545<=x<2.54 else '0'for x in df['Exoplanet Radii (EAU)']]

# Defining Variables
p_radius = df['Exoplanet Radii (EAU)']
s_radius = df['stellar_radius[R_sun]']
p_temperature = df['planet_equilibrium_temperature[K]']
stellar_temperature = df['Stellar Temperature']
p_orbit = df['planet_orbit_period[days]']
Catalog_ID = df["Tess ID"]

luminosity = s_radius**2 * (stellar_temperature/5780)**4
df['Luminosity'] = luminosity
semi_major_axis = np.cbrt((p_orbit/365)**2)
df['Semi-major Axis'] = semi_major_axis
s_flux = luminosity/semi_major_axis**2
df['Stellar Flux'] = s_flux

#Calculating ESI
result1 = ((s_flux - 1)/(s_flux +1))**2
result2 = ((p_radius - 1)/(p_radius + 1))**2
result3 = result1 + result2
result4 = (1/2)*result3
result5 = np.sqrt(result4)
ESI = 1 - result5
df['ESI'] = ESI

#Spectral Type
def condition(x):
    if x in range(10000,30000):
        return 'B'
    elif x in range(7500,10000):
        return 'A'
    elif x in range(6000,7500):
        return 'F'
    elif x in range(5200,6000):
        return 'G'
    elif x in range(3700,5200):
        return 'K'
    elif x in range(2000,3700):
        return 'M'
    elif x in range(1300,2000):
        return 'L'
df['Stellar Spectral Type'] = df['Stellar Temperature'].apply(condition)

# Exoplanet Size Range
p_size = np.empty(5858, '<U8')
x1 = np.where((df['Exoplanet Radii (EAU)']>=.495) & (df['Exoplanet Radii (EAU)']<=1.1))[0]
x2 = np.where((df['Exoplanet Radii (EAU)']>1.1) & (df['Exoplanet Radii (EAU)']<=1.754))[0]
x3 = np.where((df['Exoplanet Radii (EAU)']>1.754) & (df['Exoplanet Radii (EAU)']<=3.54))[0]
x4 = np.where((df['Exoplanet Radii (EAU)']>3.54) & (df['Exoplanet Radii (EAU)']<6))[0]
x5 = np.where((df['Exoplanet Radii (EAU)']>6))
p_size[x1] = 'Subterran'
p_size[x2] = 'Super-Earth'
p_size[x3] = 'Sub-Neptune'
p_size[x4] = 'Sub-Jovian'
p_size[x5] = 'Jovian'
df['Exoplanet Size'] = p_size

# HZ Calculation
recentVenus_seffsun = 1.776
recentVenus_a = 2.136e-4
recentVenus_b = 2.533e-8
recentVenus_c = -1.332e-11
recentVenus_d = -3.097e-15

recentMars_seffsun = .32
recentMars_a = 5.547e-5
recentMars_b = 1.526e-9
recentMars_c = -2.874e-12
recentMars_d = -5.011e-16

t = df['Stellar Temperature'] - 5780
df['t']=t

recentVenus_Seff = recentVenus_seffsun + (recentVenus_a*t) + (recentVenus_b*t**2)
+ (recentVenus_c*t**3) + (recentVenus_d*t**4)

df['RecentVenus_Seff'] = recentVenus_Seff

recentVenus_dist = ((df['Luminosity']/1)/recentVenus_Seff)**.5

recentMars_Seff = recentMars_seffsun + (recentMars_a*t) + (recentMars_b*t**2)
+ (recentMars_c*t**3) + (recentMars_d*t**4)

df['RecentMars_Seff'] = recentMars_Seff
df['RecentMars_Seff'] = recentMars_Seff
recentMars_dist = ((df['Luminosity']/1)/recentMars_Seff)**.5
df['HZ_inner'] = recentVenus_dist
df['HZ_outer'] = recentMars_dist
a = df['Semi-major Axis'].to_numpy()

Inner = a > df['HZ_inner']
Outer = a < df['HZ_outer']
b = Inner == 1
c = Outer == 1
b == c
df['HZ'] = b==c

#Filtering Stages
df1 = df[df['Exoplanet Radii (EAU)']>= .5]
df1 = df[df['Exoplanet Radii (EAU)']<= 2.54]
df2 = df1[df1["ESI"] >= .495]
df2 = df2.sort_values(by = 'ESI', ascending=[False])
df3 = df2[df2.HZ != False]

#Command for Downloading Excel File Output
Filtered_TESS_DF = df3.to_excel(r'C:\Users\emily\Documents\THEC Export.xlsx', index=False)

print('Congratulations, your data has been filtered succesfully! Download "TESS export.xlsx" on the file tab to access.')

Enter the path to the CSV file: /content/TOI_2023.05.09_20.45.09.csv
Congratulations, your data has been filtered succesfully! Download "TESS export.xlsx" on the file tab to access.
